In [ ]:
# Cài đặt các thư viện cần thiết
%pip install numpy pandas matplotlib seaborn scikit-learn


In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import os

# Cài đặt style cho biểu đồ
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

# Tạo thư mục để lưu trữ dữ liệu và hình ảnh
if not os.path.exists('processed_data'):
    os.makedirs('processed_data')
if not os.path.exists('visualizations'):
    os.makedirs('visualizations')


In [ ]:
# Định nghĩa hàm đọc và khám phá dữ liệu
def load_and_explore_data(file_path):
    """
    Đọc file dữ liệu CSV và hiển thị thông tin cơ bản
    """
    print(f"Đọc file: {file_path}")
    df = pd.read_csv(file_path)
    
    print(f"Kích thước dữ liệu: {df.shape}")
    print("\nThông tin cơ bản:")
    print(df.info())
    print("\nThống kê mô tả:")
    display(df.describe())
    print("\nKiểm tra giá trị thiếu:")
    print(df.isnull().sum())
    print("\n5 dòng đầu tiên:")
    display(df.head())
    
    return df


In [ ]:
# Tải dữ liệu từ Google Colab
from google.colab import files

print("Vui lòng tải lên tập dữ liệu bệnh tim (heart.csv, heart_disease_uci.csv và heart_cleveland_upload.csv)")
uploaded = files.upload()

# Đọc và khám phá từng tập dữ liệu
heart_df = load_and_explore_data('heart.csv')
cleveland_df = load_and_explore_data('heart_cleveland_upload.csv')
uci_df = load_and_explore_data('heart_disease_uci.csv')


In [ ]:
### 2.1 Hiển thị phân bố các đặc trưng bằng Histogram

def plot_histograms(df, dataset_name):
    """
    Vẽ histogram cho các đặc trưng số
    """
    # Lấy danh sách các cột số
    numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
    
    # Tạo lưới biểu đồ
    n_features = len(numeric_features)
    n_cols = 3
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 5))
    axes = axes.flatten()
    
    # Vẽ histogram cho từng đặc trưng
    for i, feature in enumerate(numeric_features):
        sns.histplot(df[feature], bins=20, kde=True, ax=axes[i])
        axes[i].set_title(f'Phân bố của {feature}')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Tần suất')
    
    # Ẩn các trục thừa
    for i in range(n_features, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle(f'Phân bố các đặc trưng - {dataset_name}', fontsize=16, y=1.02)
    plt.savefig(f'visualizations/{dataset_name}_histograms.png')
    plt.show()

# Vẽ histogram cho từng tập dữ liệu
print("Biểu đồ Histogram cho tập dữ liệu Heart:")
plot_histograms(heart_df, 'heart')


In [ ]:
print("Biểu đồ Histogram cho tập dữ liệu Cleveland:")
plot_histograms(cleveland_df, 'cleveland')


In [ ]:
print("Biểu đồ Histogram cho tập dữ liệu UCI:")
plot_histograms(uci_df, 'uci')


In [ ]:
### 2.2 Phân tích mối quan hệ giữa các đặc trưng bằng Scatter Plot

def plot_scatter_relationships(df, dataset_name):
    """
    Vẽ scatter plots giữa các cặp đặc trưng quan trọng
    """
    # Lựa chọn một số đặc trưng quan trọng
    key_features = []
    
    # Xác định đặc trưng đích
    target_col = 'target' if 'target' in df.columns else 'condition'
    
    # Xác định các đặc trưng số quan trọng dựa trên tập dữ liệu
    if 'age' in df.columns and 'thalach' in df.columns:
        key_features.append(('age', 'thalach'))
    
    if 'age' in df.columns and 'chol' in df.columns:
        key_features.append(('age', 'chol'))
        
    if 'thalach' in df.columns and 'chol' in df.columns:
        key_features.append(('thalach', 'chol'))
    
    if 'trestbps' in df.columns and 'chol' in df.columns:
        key_features.append(('trestbps', 'chol'))
    
    if len(key_features) == 0:
        print(f"Không tìm thấy các cặp đặc trưng phù hợp trong tập dữ liệu {dataset_name}")
        return
    
    # Vẽ scatter plots
    fig, axes = plt.subplots(1, len(key_features), figsize=(6*len(key_features), 5))
    if len(key_features) == 1:
        axes = [axes]
    
    for i, (x_feature, y_feature) in enumerate(key_features):
        sns.scatterplot(
            data=df,
            x=x_feature,
            y=y_feature,
            hue=target_col,
            palette=['blue', 'red'],
            alpha=0.7,
            ax=axes[i]
        )
        axes[i].set_title(f'{y_feature} vs {x_feature}')
        axes[i].set_xlabel(x_feature)
        axes[i].set_ylabel(y_feature)
        if i == len(key_features) - 1:  # Chỉ hiển thị legend cho biểu đồ cuối cùng
            axes[i].legend(['Không bệnh', 'Bệnh tim'], title=target_col)
    
    plt.tight_layout()
    plt.suptitle(f'Mối quan hệ giữa các đặc trưng - {dataset_name}', fontsize=16, y=1.05)
    plt.savefig(f'visualizations/{dataset_name}_scatter_plots.png')
    plt.show()

# Vẽ scatter plots cho từng tập dữ liệu
print("Biểu đồ Scatter cho tập dữ liệu Heart:")
plot_scatter_relationships(heart_df, 'heart')


In [ ]:
print("Biểu đồ Scatter cho tập dữ liệu Cleveland:")
plot_scatter_relationships(cleveland_df, 'cleveland')


In [ ]:
print("Biểu đồ Scatter cho tập dữ liệu UCI:")
plot_scatter_relationships(uci_df, 'uci')


In [ ]:
### 2.3 Phân tích tương quan bằng Heatmap

def plot_correlation_matrix(df, dataset_name):
    """
    Vẽ ma trận tương quan giữa các đặc trưng
    """
    # Lấy các cột số để tính tương quan
    numeric_df = df.select_dtypes(include=['int64', 'float64'])
    
    # Tính ma trận tương quan
    corr = numeric_df.corr()
    
    # Vẽ heatmap
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr, dtype=bool))  # Để chỉ hiển thị nửa dưới của ma trận
    sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
                square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    
    plt.title(f'Ma trận tương quan - {dataset_name}', fontsize=16)
    plt.tight_layout()
    plt.savefig(f'visualizations/{dataset_name}_correlation.png')
    plt.show()
    
    # Tìm các cặp đặc trưng có tương quan cao
    high_corr = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    high_corr_pairs = [(corr.index[i], corr.columns[j], corr.iloc[i, j]) 
                      for i, j in zip(*np.where(np.abs(high_corr) > 0.5))]
    
    if high_corr_pairs:
        print(f"\nCác cặp đặc trưng có tương quan cao (|r| > 0.5) trong tập dữ liệu {dataset_name}:")
        for feat1, feat2, corr_val in sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True):
            print(f"- {feat1} và {feat2}: {corr_val:.3f}")
    else:
        print(f"\nKhông có cặp đặc trưng nào có tương quan cao (|r| > 0.5) trong tập dữ liệu {dataset_name}")

# Vẽ ma trận tương quan cho từng tập dữ liệu
print("Ma trận tương quan cho tập dữ liệu Heart:")
plot_correlation_matrix(heart_df, 'heart')


In [ ]:
print("\nMa trận tương quan cho tập dữ liệu Cleveland:")
plot_correlation_matrix(cleveland_df, 'cleveland')


In [ ]:
print("\nMa trận tương quan cho tập dữ liệu UCI:")
plot_correlation_matrix(uci_df, 'uci')


In [ ]:
### 2.4 Phân tích xu hướng bằng Line Chart

def plot_line_charts(df, dataset_name):
    """
    Vẽ biểu đồ đường phân tích xu hướng theo tuổi
    """
    # Xác định biến đích
    target_col = 'target' if 'target' in df.columns else 'condition'
    
    # Kiểm tra xem có cột tuổi không
    if 'age' not in df.columns:
        print(f"Không tìm thấy cột tuổi trong tập dữ liệu {dataset_name}")
        return
    
    # Tạo các nhóm tuổi
    df['age_group'] = pd.cut(df['age'], bins=range(30, 85, 5), right=False)
    
    # Tính tỉ lệ mắc bệnh theo từng nhóm tuổi
    age_disease_rate = df.groupby('age_group')[target_col].mean().reset_index()
    
    # Vẽ biểu đồ đường
    plt.figure(figsize=(12, 6))
    plt.plot(age_disease_rate['age_group'].astype(str), 
             age_disease_rate[target_col], 
             marker='o', 
             linewidth=2, 
             markersize=10)
    
    plt.title(f'Tỉ lệ mắc bệnh tim theo độ tuổi - {dataset_name}', fontsize=16)
    plt.xlabel('Nhóm tuổi')
    plt.ylabel('Tỉ lệ mắc bệnh tim')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'visualizations/{dataset_name}_age_disease_trend.png')
    plt.show()
    
    # Nếu có dữ liệu về cholesterol, vẽ xu hướng theo cholesterol
    if 'chol' in df.columns:
        # Tạo các nhóm cholesterol
        df['chol_group'] = pd.cut(df['chol'], bins=range(100, 601, 50), right=False)
        
        # Tính tỉ lệ mắc bệnh theo từng nhóm cholesterol
        chol_disease_rate = df.groupby('chol_group')[target_col].mean().reset_index()
        
        # Vẽ biểu đồ đường
        plt.figure(figsize=(12, 6))
        plt.plot(chol_disease_rate['chol_group'].astype(str), 
                chol_disease_rate[target_col], 
                marker='o', 
                linewidth=2,
                color='red',
                markersize=10)
        
        plt.title(f'Tỉ lệ mắc bệnh tim theo mức Cholesterol - {dataset_name}', fontsize=16)
        plt.xlabel('Nhóm Cholesterol (mg/dl)')
        plt.ylabel('Tỉ lệ mắc bệnh tim')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'visualizations/{dataset_name}_chol_disease_trend.png')
        plt.show()

# Vẽ line charts cho từng tập dữ liệu
print("Biểu đồ Line Chart cho tập dữ liệu Heart:")
plot_line_charts(heart_df, 'heart')


In [ ]:
print("\nBiểu đồ Line Chart cho tập dữ liệu Cleveland:")
plot_line_charts(cleveland_df, 'cleveland')


In [ ]:
print("\nBiểu đồ Line Chart cho tập dữ liệu UCI:")
plot_line_charts(uci_df, 'uci')


In [ ]:
### 3.1 Xử lý giá trị thiếu và chuẩn hóa dữ liệu Heart

def preprocess_heart_data(df):
    """
    Tiền xử lý dữ liệu từ tập Heart
    """
    print("Đang xử lý dữ liệu Heart...")
    
    # Tạo bản sao để không ảnh hưởng đến dữ liệu gốc
    data = df.copy()
    
    # Kiểm tra và xử lý giá trị thiếu
    if data.isnull().sum().sum() > 0:
        print("Đang xử lý giá trị thiếu...")
        numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
        numeric_imputer = SimpleImputer(strategy='median')
        data[numeric_features] = numeric_imputer.fit_transform(data[numeric_features])
    else:
        print("Không có giá trị thiếu trong dữ liệu Heart")
    
    # Chuẩn hóa các đặc trưng số
    print("Đang chuẩn hóa các đặc trưng số...")
    features = data.drop('target', axis=1)
    target = data['target']
    
    # Hiển thị thông tin về đặc trưng và nhãn
    print(f"\nSố lượng mẫu: {data.shape[0]}")
    print(f"Số lượng đặc trưng: {features.shape[1]}")
    print(f"Phân bố nhãn: {target.value_counts().to_dict()}")
    
    return data

# Xử lý dữ liệu Heart
heart_processed = preprocess_heart_data(heart_df)
print("\nDữ liệu Heart sau khi tiền xử lý:")
display(heart_processed.head())


In [ ]:
### 3.2 Xử lý giá trị thiếu và chuẩn hóa dữ liệu Cleveland

def preprocess_cleveland_data(df):
    """
    Tiền xử lý dữ liệu từ tập Cleveland
    """
    print("Đang xử lý dữ liệu Cleveland...")
    
    # Tạo bản sao để không ảnh hưởng đến dữ liệu gốc
    data = df.copy()
    
    # Đổi tên cột condition thành target cho thống nhất (nếu có)
    if 'condition' in data.columns:
        data.rename(columns={'condition': 'target'}, inplace=True)
        print("Đã đổi tên cột 'condition' thành 'target' để thống nhất")
    
    # Chuyển đổi nhãn thành dạng nhị phân (0: không bệnh, 1: có bệnh)
    if 'target' in data.columns and data['target'].max() > 1:
        print("Chuyển đổi nhãn thành dạng nhị phân (0/1)...")
        data['target'] = data['target'].apply(lambda x: 0 if x == 0 else 1)
    
    # Kiểm tra và xử lý giá trị thiếu
    if data.isnull().sum().sum() > 0:
        print("Đang xử lý giá trị thiếu...")
        numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
        numeric_imputer = SimpleImputer(strategy='median')
        data[numeric_features] = numeric_imputer.fit_transform(data[numeric_features])
    else:
        print("Không có giá trị thiếu trong dữ liệu Cleveland")
    
    # Hiển thị thông tin về đặc trưng và nhãn
    features = data.drop('target', axis=1)
    target = data['target']
    print(f"\nSố lượng mẫu: {data.shape[0]}")
    print(f"Số lượng đặc trưng: {features.shape[1]}")
    print(f"Phân bố nhãn: {target.value_counts().to_dict()}")
    
    return data

# Xử lý dữ liệu Cleveland
cleveland_processed = preprocess_cleveland_data(cleveland_df)
print("\nDữ liệu Cleveland sau khi tiền xử lý:")
display(cleveland_processed.head())


In [ ]:
### 3.3 Xử lý giá trị thiếu và chuẩn hóa dữ liệu UCI

def preprocess_uci_data(df):
    """
    Tiền xử lý dữ liệu từ tập UCI
    """
    print("Đang xử lý dữ liệu UCI...")
    
    # Tạo bản sao để không ảnh hưởng đến dữ liệu gốc
    data = df.copy()
    
    # Xử lý giá trị thiếu
    if data.isnull().sum().sum() > 0:
        print("Đang xử lý giá trị thiếu...")
        # Xử lý các cột số
        numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
        numeric_imputer = SimpleImputer(strategy='median')
        data[numeric_features] = numeric_imputer.fit_transform(data[numeric_features])
        
        # Xử lý các cột phân loại
        categorical_features = data.select_dtypes(include=['object']).columns
        if len(categorical_features) > 0:
            categorical_imputer = SimpleImputer(strategy='most_frequent')
            data[categorical_features] = categorical_imputer.fit_transform(data[categorical_features])
            print(f"Đã xử lý giá trị thiếu trong {len(categorical_features)} cột phân loại")
    else:
        print("Không có giá trị thiếu trong dữ liệu UCI")
    
    # Xử lý cột target (nếu có cột 'num')
    if 'num' in data.columns:
        # Chuyển thành dạng nhị phân: 0 = không bệnh (0), 1 = có bệnh (1-4)
        data['target'] = data['num'].apply(lambda x: 0 if x == 0 else 1)
        data = data.drop('num', axis=1)
        print("Đã tạo cột 'target' từ cột 'num' và chuyển về dạng nhị phân (0/1)")
    
    # Loại bỏ các cột không cần thiết
    if 'id' in data.columns:
        data = data.drop('id', axis=1)
        print("Đã loại bỏ cột 'id'")
    
    # Mã hóa các biến phân loại
    categorical_features = data.select_dtypes(include=['object']).columns
    if len(categorical_features) > 0:
        print(f"Đang mã hóa {len(categorical_features)} biến phân loại...")
        # One-hot encoding cho các biến phân loại
        encoded_data = pd.get_dummies(data[categorical_features], drop_first=True)
        
        # Loại bỏ các cột phân loại gốc và thêm các cột đã mã hóa
        data = data.drop(categorical_features, axis=1)
        data = pd.concat([data, encoded_data], axis=1)
    
    # Hiển thị thông tin về đặc trưng và nhãn
    if 'target' in data.columns:
        features = data.drop('target', axis=1)
        target = data['target']
        print(f"\nSố lượng mẫu: {data.shape[0]}")
        print(f"Số lượng đặc trưng: {features.shape[1]}")
        print(f"Phân bố nhãn: {target.value_counts().to_dict()}")
    else:
        print("\nKhông tìm thấy cột 'target' trong dữ liệu UCI sau khi tiền xử lý")
    
    return data

# Xử lý dữ liệu UCI
uci_processed = preprocess_uci_data(uci_df)
print("\nDữ liệu UCI sau khi tiền xử lý:")
display(uci_processed.head())


In [ ]:
def split_and_scale_data(df, dataset_name):
    """
    Chia dữ liệu thành tập huấn luyện và tập kiểm tra, sau đó chuẩn hóa
    """
    print(f"Đang chia và chuẩn hóa dữ liệu {dataset_name}...")
    
    # Xác định biến đích
    if 'target' not in df.columns:
        print(f"Không tìm thấy cột 'target' trong dữ liệu {dataset_name}")
        return None, None, None, None, None
    
    # Tách đặc trưng và nhãn
    X = df.drop('target', axis=1)
    y = df['target']
    
    # Chia thành tập huấn luyện và tập kiểm tra (80% - 20%)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Hiển thị thông tin về kích thước dữ liệu
    print(f"Kích thước tập huấn luyện: {X_train.shape[0]} mẫu, {X_train.shape[1]} đặc trưng")
    print(f"Kích thước tập kiểm tra: {X_test.shape[0]} mẫu, {X_test.shape[1]} đặc trưng")
    print(f"Phân bố nhãn tập huấn luyện: {y_train.value_counts().to_dict()}")
    print(f"Phân bố nhãn tập kiểm tra: {y_test.value_counts().to_dict()}")
    
    return X_train, X_test, y_train, y_test, scaler


In [ ]:
# Chia và chuẩn hóa dữ liệu Heart
print("=== Chia dữ liệu Heart ===")
heart_X_train, heart_X_test, heart_y_train, heart_y_test, heart_scaler = split_and_scale_data(heart_processed, 'Heart')

# Chia và chuẩn hóa dữ liệu Cleveland
print("\n=== Chia dữ liệu Cleveland ===")
cleveland_X_train, cleveland_X_test, cleveland_y_train, cleveland_y_test, cleveland_scaler = split_and_scale_data(cleveland_processed, 'Cleveland')

# Chia và chuẩn hóa dữ liệu UCI
print("\n=== Chia dữ liệu UCI ===")
uci_X_train, uci_X_test, uci_y_train, uci_y_test, uci_scaler = split_and_scale_data(uci_processed, 'UCI')


In [ ]:
def plot_scaled_data_distribution(X_train_scaled, X_test_scaled, feature_names, dataset_name):
    """
    Vẽ biểu đồ phân bố của dữ liệu đã chuẩn hóa
    """
    # Chọn một số đặc trưng quan trọng để hiển thị
    n_features = min(5, len(feature_names))
    selected_features = list(range(n_features))
    
    # Chuyển dữ liệu đã chuẩn hóa về dạng DataFrame để dễ thao tác
    X_train_df = pd.DataFrame(X_train_scaled[:, selected_features], columns=feature_names[selected_features])
    X_test_df = pd.DataFrame(X_test_scaled[:, selected_features], columns=feature_names[selected_features])
    
    # Thêm cột để đánh dấu tập dữ liệu
    X_train_df['dataset'] = 'Train'
    X_test_df['dataset'] = 'Test'
    
    # Gộp dữ liệu để vẽ biểu đồ
    combined_df = pd.concat([X_train_df, X_test_df])
    
    # Chuyển dữ liệu về dạng "long form" để vẽ biểu đồ
    melted_df = pd.melt(combined_df, id_vars=['dataset'], var_name='Feature', value_name='Value')
    
    # Vẽ biểu đồ phân bố
    plt.figure(figsize=(15, 10))
    sns.boxplot(x='Feature', y='Value', hue='dataset', data=melted_df)
    plt.title(f'Phân bố dữ liệu đã chuẩn hóa - {dataset_name}', fontsize=16)
    plt.xlabel('Đặc trưng', fontsize=12)
    plt.ylabel('Giá trị (đã chuẩn hóa)', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'visualizations/{dataset_name}_scaled_distribution.png')
    plt.show()

# Vẽ phân bố dữ liệu đã chuẩn hóa cho tập Heart
if heart_X_train is not None and heart_X_test is not None:
    heart_feature_names = np.array(heart_processed.drop('target', axis=1).columns)
    print("Phân bố dữ liệu Heart sau khi chuẩn hóa:")
    plot_scaled_data_distribution(heart_X_train, heart_X_test, heart_feature_names, 'Heart')


In [ ]:
# Vẽ phân bố dữ liệu đã chuẩn hóa cho tập Cleveland
if cleveland_X_train is not None and cleveland_X_test is not None:
    cleveland_feature_names = np.array(cleveland_processed.drop('target', axis=1).columns)
    print("\nPhân bố dữ liệu Cleveland sau khi chuẩn hóa:")
    plot_scaled_data_distribution(cleveland_X_train, cleveland_X_test, cleveland_feature_names, 'Cleveland')

# Vẽ phân bố dữ liệu đã chuẩn hóa cho tập UCI
if uci_X_train is not None and uci_X_test is not None:
    uci_feature_names = np.array(uci_processed.drop('target', axis=1).columns)
    print("\nPhân bố dữ liệu UCI sau khi chuẩn hóa:")
    plot_scaled_data_distribution(uci_X_train, uci_X_test, uci_feature_names, 'UCI')


In [ ]:
def combine_datasets(datasets, dataset_names):
    """
    Kết hợp nhiều tập dữ liệu thành một
    """
    # Kiểm tra xem có đủ dữ liệu không
    if len(datasets) < 2:
        print("Cần ít nhất 2 tập dữ liệu để kết hợp")
        return None
    
    # Tìm các cột chung giữa các tập dữ liệu
    common_columns = set.intersection(*[set(df.columns) for df in datasets])
    
    # Kiểm tra xem có cột 'target' không
    if 'target' not in common_columns:
        print("Cột 'target' không có trong tất cả các tập dữ liệu, không thể kết hợp!")
        return None
    
    print(f"Các cột chung giữa các tập dữ liệu: {common_columns}")
    
    # Chỉ giữ lại các cột chung và kết hợp dữ liệu
    combined_data = pd.concat([df[list(common_columns)] for df in datasets], ignore_index=True)
    
    # Thêm cột để đánh dấu nguồn dữ liệu
    for i, (df, name) in enumerate(zip(datasets, dataset_names)):
        combined_data.loc[combined_data.index.isin(range(len(combined_data) - len(df), len(combined_data))), 'source'] = name
    
    print(f"Đã kết hợp {len(datasets)} tập dữ liệu với {combined_data.shape[0]} mẫu và {len(common_columns)} đặc trưng chung")
    
    return combined_data

# Thử kết hợp các tập dữ liệu
datasets_to_combine = []
dataset_names = []

if 'target' in heart_processed.columns:
    datasets_to_combine.append(heart_processed)
    dataset_names.append('Heart')

if 'target' in cleveland_processed.columns:
    datasets_to_combine.append(cleveland_processed)
    dataset_names.append('Cleveland')

print("Kết hợp dữ liệu từ các nguồn...")
combined_df = combine_datasets(datasets_to_combine, dataset_names)

if combined_df is not None:
    print("\nDữ liệu sau khi kết hợp:")
    display(combined_df.head())
    
    # Trực quan hóa dữ liệu kết hợp
    print("\nTrực quan hóa dữ liệu kết hợp:")
    plot_histograms(combined_df, 'combined')
    plot_correlation_matrix(combined_df, 'combined')
    
    # Chia dữ liệu kết hợp
    print("\n=== Chia dữ liệu kết hợp ===")
    combined_X_train, combined_X_test, combined_y_train, combined_y_test, combined_scaler = split_and_scale_data(combined_df, 'Combined')


In [ ]:
def save_processed_data(X_train, X_test, y_train, y_test, dataset_name):
    """
    Lưu dữ liệu đã xử lý vào thư mục processed_data
    """
    if X_train is None or X_test is None or y_train is None or y_test is None:
        print(f"Không thể lưu dữ liệu {dataset_name} do thiếu dữ liệu")
        return
    
    # Tạo thư mục nếu chưa tồn tại
    if not os.path.exists('processed_data'):
        os.makedirs('processed_data')
    
    # Lưu dữ liệu
    np.save(f'processed_data/{dataset_name.lower()}_X_train.npy', X_train)
    np.save(f'processed_data/{dataset_name.lower()}_X_test.npy', X_test)
    np.save(f'processed_data/{dataset_name.lower()}_y_train.npy', y_train)
    np.save(f'processed_data/{dataset_name.lower()}_y_test.npy', y_test)
    
    print(f"Đã lưu dữ liệu {dataset_name} vào thư mục processed_data")

# Lưu dữ liệu Heart
save_processed_data(heart_X_train, heart_X_test, heart_y_train, heart_y_test, 'heart')

# Lưu dữ liệu Cleveland
save_processed_data(cleveland_X_train, cleveland_X_test, cleveland_y_train, cleveland_y_test, 'cleveland')

# Lưu dữ liệu UCI
save_processed_data(uci_X_train, uci_X_test, uci_y_train, uci_y_test, 'uci')

# Lưu dữ liệu Combined (nếu có)
if 'combined_X_train' in locals() and combined_X_train is not None:
    save_processed_data(combined_X_train, combined_X_test, combined_y_train, combined_y_test, 'combined')


In [ ]:
# Thống kê về các tập dữ liệu
data_stats = pd.DataFrame({
    'Tập dữ liệu': ['Heart', 'Cleveland', 'UCI', 'Combined'],
    'Số lượng mẫu': [
        heart_processed.shape[0] if 'heart_processed' in locals() else 0,
        cleveland_processed.shape[0] if 'cleveland_processed' in locals() else 0,
        uci_processed.shape[0] if 'uci_processed' in locals() else 0,
        combined_df.shape[0] if 'combined_df' in locals() else 0
    ],
    'Số lượng đặc trưng': [
        heart_processed.shape[1] - 1 if 'heart_processed' in locals() else 0,  # Trừ đi cột target
        cleveland_processed.shape[1] - 1 if 'cleveland_processed' in locals() else 0,
        uci_processed.shape[1] - 1 if 'uci_processed' in locals() else 0,
        combined_df.shape[1] - 1 if 'combined_df' in locals() else 0
    ],
    'Tỷ lệ mắc bệnh': [
        heart_processed['target'].mean() if 'heart_processed' in locals() else 0,
        cleveland_processed['target'].mean() if 'cleveland_processed' in locals() else 0,
        uci_processed['target'].mean() if 'uci_processed' in locals() else 0,
        combined_df['target'].mean() if 'combined_df' in locals() else 0
    ]
})

# Hiển thị bảng thống kê
print("Tổng kết về các tập dữ liệu:")
display(data_stats)

# Tạo biểu đồ so sánh
plt.figure(figsize=(12, 8))
sns.barplot(x='Tập dữ liệu', y='Số lượng mẫu', data=data_stats)
plt.title('So sánh kích thước các tập dữ liệu', fontsize=16)
plt.ylabel('Số lượng mẫu')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig('visualizations/datasets_size_comparison.png')
plt.show()

# Vẽ biểu đồ tỷ lệ mắc bệnh
plt.figure(figsize=(12, 8))
sns.barplot(x='Tập dữ liệu', y='Tỷ lệ mắc bệnh', data=data_stats)
plt.title('So sánh tỷ lệ mắc bệnh giữa các tập dữ liệu', fontsize=16)
plt.ylabel('Tỷ lệ mắc bệnh (%)')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig('visualizations/disease_rate_comparison.png')
plt.show()

print("\nQuá trình tiền xử lý dữ liệu đã hoàn tất!")
print("- Dữ liệu đã được làm sạch, xử lý giá trị thiếu và chuẩn hóa")
print("- Đã chia tập dữ liệu thành tập huấn luyện (80%) và tập kiểm tra (20%)")
print("- Các tập dữ liệu đã được lưu vào thư mục processed_data để sử dụng cho việc huấn luyện mô hình")
print("- Các biểu đồ trực quan hóa đã được lưu vào thư mục visualizations")
